## Cindey Caine A10834042
## CSE 151 PA1

In [85]:
df

,,K = 1,K = 3,K = 9,K = 15
0,Training Error,0.000,0.055,0.072,0.0935
1,Validation Error,0.082,0.097,0.104,0.1030


## 1. The K which performs best on validation data is K = 1. The test error for K = 1 is:

In [86]:
print("Test error for when K = 1 is: " + str(testError[bestK]))

Test error for when K = 1 is: 0.094


In [87]:
df2

,,K = 1,K = 3,K = 9,K = 15
0,Projection Training Error,0.00,0.194,0.2295,0.253
1,Projection Validation Error,0.32,0.297,0.2940,0.287


## 2. The K which performs best on projection validation data is K = 9. The test error for K = 9 is:

In [88]:
print("Test error for when K = 9 is: " + str(testErrorM[bestKm]))

Test error for when K = 9 is: 0.298


## The classification accuracy decreases when using the projection matrix. This is because the number of labels is reduced from 748 to 20. 

In [93]:
dfTime

,,Test Time,Training Time,Validation Time
0,Original Labels,209.719450,420.044135,210.667919
1,Projection Matrix Applied,18.236767,36.585633,18.231530


## However, the running time drastically changes when using the projection matrix on the original data. The algorithm takes much less time to run because it only needs to consider the distances of the 20 labels per row. 

In [92]:
dfTime = pd.DataFrame(data=dTime)

In [90]:
# Display dataframe of all the time taken to run the KNN algorithm in seconds
dTime = {' ': ['Original Labels', 'Projection Matrix Applied'], 'Training Time': [trainTime,trainTimeM],
         'Validation Time': [valTime,valTimeM],'Test Time': [testTime,testTimeM]}

In [79]:
# Create a dataframe for displaying the Training Error/Validation Error
d = {' ':['Training Error', 'Validation Error'], 'K = 1':[trainError[0],valError[0][1]],
     'K = 3':[trainError[1],valError[1][1]],'K = 9':[trainError[2],valError[2][1]]}

In [80]:
df = pd.DataFrame(data=d,index=None)
df['K = 15'] = [trainError[3],valError[3][1]]

In [81]:
# Create a dataframe for displaying the Training Error/Validation Error on Proj Mat
d2 = {' ':['Projection Training Error', 'Projection Validation Error'], 'K = 1':[trainErrorM[0],valErrorM[0][1]],
     'K = 3':[trainErrorM[1],valErrorM[1][1]],'K = 9':[trainErrorM[2],valErrorM[2][1]]}

In [83]:
df2 = pd.DataFrame(data=d2,index=None)
df2['K = 15'] = [trainErrorM[3],valErrorM[3][1]]

In [39]:
import numpy as np
import pandas as pd
import time
from random import randint
from numpy import linalg as LA
from collections import Counter

In [2]:
# Read in data
with open('../pa1train.txt') as f:
    train = f.readlines()
train = [x.strip() for x in train] 

with open('../pa1validate.txt') as f:
    val = f.readlines()
val = [x.strip() for x in val] 

with open('../pa1test.txt') as f:
    test = f.readlines()
test = [x.strip() for x in test] 

In [3]:
# Read in proj matrix and remove spacing
with open('../projection.txt') as f:
    proj = f.readlines()
proj = [x.strip() for x in proj] 

for i in range (0,len(proj)):
    proj[i] = proj[i].split(" ")
    
for i in range(0,len(proj)):
    proj[i] = [float(j) for j in proj[i]]

In [4]:
# Apply Matrix Multiplication to creat projection matrix
def matrixMult(data,proj):
    # create empty lists
    projMat = []
    projRow = []
    x = 0
    for i in range (0,len(data)):
        for k in range (0,len(proj[0])):
            for j in range (0, len(proj)):
                x += (data[i][j] * proj[j][k])
            projRow.append(x)
            x = 0
        projMat.append(projRow)
        projRow = []
    return projMat

In [5]:
def KNN(training, toPred, biggestkVal):
    allDist = []
    kResults = []

    # calculate all the distances between vector to predict and training data
    for i in range (0,len(training)):
        dist = LA.norm(np.array(training[i])-np.array(toPred))
        allDist.append((i,dist))
    
    # sort all values by distance
    allDist.sort(key=lambda tup: tup[1]) 

    # check for ties
    i = 0
    while(len(kResults) != biggestkVal):
        # if there is a tie, randomly choose one of the values
        if(allDist[i][1] == allDist[i+1][1]):
            coin = randint(i,i+1)
            kResults.append(allDist[coin][0])
        else:
            kResults.append(allDist[i][0])
        i += 1
    
    return kResults    

In [6]:
# function to generate predictions given training data, training labels, test data, and greatest k value
def kNNPred(trainSet, trainL, testSet, largeK):
    pred = []
    totPred = []
    for i in range (0,len(testSet)):
        dup = 0
    
        # outputs largest K values of training data predictions on one test data point
        output = KNN(trainSet,testSet[i],largeK)
    
        for m in range (0,len(k)):
            # get the current k value
            curK = k[m]
        
            # counts label occurences in the predictions
            data = Counter([trainL[x] for x in output[:curK]])
    
            # if there is only one common label append it
            if(len(data.most_common()) == 1):
                pred.append(data.most_common()[0][0])
            else:
            # check all top common data for duplicates
                for j in range (0,len(data.most_common())-1):
                    # if current top count == next top count then add duplicates
                    if(data.most_common()[j][1] == data.most_common()[j+1][1]):
                        dup += 1
                    else:
                        break
                # pick a random duplicate
                pred.append(data.most_common()[randint(0,dup)][0])
            dup = 0
        totPred.append(pred)
        pred = []
    return totPred

In [7]:
# calculate error given labels, predictions, and number of k values
def calcError(labels, predictions,numK):
    error = []
    allCount = []
    counter = 0
    for j in range (0, numK):
        for i in range (0,len(labels)):
            if(labels[i] == predictions[i][j]):
                counter += 1
        allCount.append(counter)
        counter = 0
    for i in range (0,len(allCount)):
        error.append((len(labels)-allCount[i])/len(labels))
    return error

In [8]:
# create empty lists for data sorting
trainData = []
trainLabel = []

valData = []
valLabel = []

testData = []
testLabel = []

In [9]:
# Remove spaces and split into label/data
for i in range (0,len(train)):
    train[i] = train[i].split(" ")
    trainLabel.append(train[i][784])
    trainData.append(train[i][:784])

for i in range (0,len(val)):
    val[i] = val[i].split(" ")
    valLabel.append(val[i][784])
    valData.append(val[i][:784])

for i in range (0,len(test)):
    test[i] = test[i].split(" ")
    testLabel.append(test[i][784])
    testData.append(test[i][:784])

In [10]:
# convert label string to int
trainLabel = [int(i) for i in trainLabel]

valLabel = [int(i) for i in valLabel]

testLabel = [int(i) for i in testLabel]

In [11]:
# convert data string to int
for i in range(0,len(trainData)):
    trainData[i] = [int(j) for j in trainData[i]]
    
for i in range(0,len(valData)):
    valData[i] = [int(j) for j in valData[i]]
    
for i in range(0,len(testData)):
    testData[i] = [int(j) for j in testData[i]]

In [12]:
# init projection matrixes
trainProj = []
valProj = []
testProj = []

In [13]:
# calculate all projection matrix mult on train/val/test data
trainProj = matrixMult(trainData,proj)
valProj = matrixMult(valData,proj)
testProj = matrixMult(testData,proj)

In [14]:
# create list of k values
k = [1,5,9,15]

In [15]:
# Prediction Lists
trainPred = []
valPred = []
testPred = []

In [16]:
# Train with training data
t0 = time.time() 
trainPred = kNNPred(trainData,trainLabel,trainData,15)
t1 = time.time()
trainTime = t1-t0

In [17]:
trainError = calcError(trainLabel,trainPred,len(k))

In [18]:
trainError

[0.0, 0.055, 0.072, 0.0935]

In [19]:
# Train with validation data
t0 = time.time() 
valPred = kNNPred(trainData,trainLabel,valData,15)
t1 = time.time()
valTime = t1-t0

In [20]:
valError = calcError(valLabel,valPred,len(k))

In [21]:
# Find best k with the lowest error
for i in range (0,len(valError)):
    valError[i] = (i,valError[i])
bestK = min(valError, key = lambda t: t[1])[0]

In [22]:
# Train with Test data
t0 = time.time() 
testPred = kNNPred(trainData,trainLabel,testData,15)
t1 = time.time()
testTime = t1-t0

In [23]:
testError = calcError(testLabel,testPred,len(k))

In [24]:
testError[bestK]

0.094

In [25]:
# Prediction Lists
trainPredM = []
valPredM = []
testPredM = []

In [26]:
# Train with training data projection
t0 = time.time() 
trainPredM = kNNPred(trainProj,trainLabel,trainProj,15)
t1 = time.time()
trainTimeM = t1-t0

In [27]:
trainErrorM = calcError(trainLabel,trainPredM,len(k))

In [28]:
# Train with validation data projection
t0 = time.time() 
valPredM = kNNPred(trainProj,trainLabel,valProj,15)
t1 = time.time()
valTimeM = t1-t0

In [29]:
valErrorM = calcError(valLabel,valPredM,len(k))

In [30]:
valErrorM

[0.32, 0.297, 0.294, 0.287]

In [31]:
# Find best k with the lowest error
for i in range (0,len(valErrorM)):
    valErrorM[i] = (i,valErrorM[i])
bestKm = min(valErrorM, key = lambda t: t[1])[0]

In [32]:
# Train with Test data
t0 = time.time() 
testPredM = kNNPred(trainProj,trainLabel,testProj,15)
t1 = time.time()
testTimeM = t1-t0

In [33]:
testErrorM = calcError(testLabel,testPredM,len(k))

In [34]:
testErrorM[bestKm]

0.298